In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.2 MB/s eta 0:00:00


In [3]:
!pip install langchain

In [4]:
!pip install langchain_community
!pip install chromadb
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Instal

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from langchain.embeddings import HuggingFaceEmbeddings

import chromadb
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma


In [6]:
loader = PyPDFLoader("knust_Admission_Requirement.pdf")
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


for doc in docs:
    print(doc.page_content)

Entry Requirements and Guidelines for Selecting an Undergraduate Programme   2024/2025   Published by The Academic Affairs Division Registrar’s Office April 2024
1
Table of Contents 1.0 STEP-BY-STEP INSTRUCTIONS ..................................................................................... 2 A. STEP 1: Purchase of E-Voucher ................................................................................................ 2 B. STEP 2: Check List ................................................................................................................... 2 C. STEP 3 Application .................................................................................................................. 3 2.0 CHOICE OF PROGRAMME OF STUDY ........................................................................... 5 LIST OF UNDERGRADUATE PROGRAMMES OFFERED BY COLLEGES .............................................. 5 3.0 ADMISSION REQUIREMENTS .................................................

In [7]:
texts = [doc.page_content for doc in docs]

In [8]:
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
embeddings = model.encode(texts)
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-8-3665c2250c36>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [9]:
client = chromadb.Client()
collection_name = "my_collection_4"
collection = client.create_collection(collection_name)

db = Chroma.from_documents(documents, embedding=hf_embeddings)


In [10]:
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    collection.add(
        documents=[text],
        embeddings=[embedding],
        ids=[str(i)]
    )

print("Embeddings successfully stored in ChromaDB.")

Embeddings successfully stored in ChromaDB.


In [11]:
def hybrid_search(query_text, top_n=5):
    # Generate embedding for the query
    query_embedding = embedding_model.encode([query_text])[0]

    # Vector-based search
    vector_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_n
    )

    # Keyword-based search
    keyword_results = collection.query(
        query_texts=[query_text],
        n_results=top_n
    )

    # Combine results
    combined_results = []
    seen = set()

    for result_list in vector_results["documents"]:
        for result in result_list:
            if result not in seen:
                combined_results.append(result)
                seen.add(result)

    for result_list in keyword_results["documents"]:
        for result in result_list:
            if result not in seen:
                combined_results.append(result)
                seen.add(result)

    return combined_results


In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


<ipython-input-13-4cbb8db76ddd>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [14]:
template = """
You are an assistant knowledgeable about university admission requirements.
Use the following conversation history and search results to provide accurate and helpful answers.

Conversation history:
{chat_history}

Search results:
{context}

User query:
{query}

Assistant response:
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "query"],
    template=template
)

In [78]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


hf_pipeline = pipeline(

    task="text2text-generation",
    model="google/flan-t5-large",
    max_new_tokens=512,
     device=0

)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})  # Using max marginal relevance (MMR)


In [79]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [80]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [92]:


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)


user_query = "Entry requirement for Engineering"
#response = qa_chain.run(user_query)
response = qa_chain.invoke(user_query)

# Display response
#print("AI Agent Response:")
print(response)